# Get Started

Setup the env and lib.

In [ ]:
#!pip install tensorflow
#!pip install keras
#!pip install seaborn
#!pip install directory_structure

In [ ]:
!conda list tensorflow-gpu

Check the notebook kernel.

In [ ]:
from jupyter_core.paths import jupyter_data_dir
print(jupyter_data_dir())

import sys
sys.executable

Get the GPU/CPU info

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
K.clear_session()
# tf.debugging.set_log_device_placement(True)

print("Here: ")
print(tf.test.is_built_with_cuda())
print()

if tf.test.gpu_device_name():
    print("#############################################################")
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    print("#############################################################")
else:
    print("Please install GPU version of TF")

Get a list of GPU.

In [ ]:
print("# GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Init the cudnn and cuda.

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

####################################################################################################################################

# Model Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adagrad, Adadelta, RMSprop, Adam, Adamax

Get the test set from the disk. Let's visually see the structure of the dir.

In [ ]:
from directory_structure import Tree

print(Tree("./dataset/test/"))

We load the test set from a given dir and normalize it. At the same time, we assign the batch size as 32 and target size as 64*64.

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    './dataset/test/',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

# Load the model(s) from the disk

## Load a list of models

Load the models as a list.<br>
<p style="color:#FF0000";>Note, DO NOT perform this unless you have enough RAM in your computer!</p>

In [ ]:
# model_trained = []

# for file in os.scandir("./"):
#     if "h5" in file.name:
#         model_trained.append({
#             "name": file.name[:-2],
#             "model": tf.keras.models.load_model(file.name),
#         })

## Load a single model

In [ ]:
model_name = "Adagrad"
loaded_model_adagrad = tf.keras.models.load_model(model_name+".h5")
print("Loaded {} model from disk".format(model_name))

# Prediction

## Single Prediction

In order to know what binary value of each class is assigned, we can simply use "train_set.class_indices"
<br>
<br>
Now we need to convert ImageDataGenerator to numpy representation.

In [ ]:
# from tensorflow.keras.preprocessing import image

# file_path = ""
# test_img = image.load_img(file_path, target_size=(64,64))
# test_img = image.img_to_array(test_img)
# test_img = np.expand_dims(test_img, axis=0)

# y_pred = loaded_model_adagrad.predict(test_set)

# train_set.class_indices

# if y_pred[0][0] == 1:
#     print("Dog")
# else:
#     print("Cat")

## Multiple Predictions

In [ ]:
y_pred = loaded_model_adagrad.predict(test_set)

# the prediction result will be contious val, so we need to apply a method that's similar to sigmoid func
y_pred = np.round_(y_pred)

In [ ]:
y_pred.shape

Convert "flow_from_directory" to numpy format. Ref: https://stackoverflow.com/questions/42284873/assign-imagedatagenerator-result-to-numpy-array

In [ ]:
y = np.concatenate([test_set.next()[1] for i in range(test_set.__len__())])

In [ ]:
y.shape

# Metrics

## Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
def plot_confusionM(y_pred, title):
    comf = confusion_matrix(y, y_pred)

    # plot the confusion metrix
    ax = sns.heatmap(comf, annot=True, cmap='Blues')

    ax.set_title(title);
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');

    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(['False','True'])
    ax.yaxis.set_ticklabels(['False','True'])

    ## Display the visualization of the Confusion Matrix.
    plt.show()

Output a detailed report about the classifier.

In [ ]:
print(classification_report(y, y_pred))